# Importación de módulos

In [75]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importación de datos

In [76]:
# Copia los datos de un documento CSV en un DataFrame.
data_df = pd.read_csv('synchronous_machine.csv')
data_df

,Iy,PF,e,dIf,If
0,3.0,0.66,0.34,0.383,1.563
1,3.0,0.68,0.32,0.372,1.552
2,3.0,0.70,0.30,0.360,1.540
3,3.0,0.72,0.28,0.338,1.518
4,3.0,0.74,0.26,0.317,1.497
...,...,...,...,...,...
552,6.0,0.91,0.09,0.142,1.322
553,6.0,0.93,0.07,0.151,1.331
554,6.0,0.95,0.05,0.160,1.340
555,6.0,0.97,0.03,0.160,1.340


# Análisis de los datos

In [77]:
'''/* 
Recopila y muestra la información necesaria para conocer si hay outliers.
*/'''

# Retorna media, desviación estándar, min, cuantiles 25,50,75% y max.
display(data_df.describe())
# Retorna si hay valores en blanco o no numericos.
display(data_df.isna().sum())

# Establece los datos necesarios y títulos de eje para mostrar varias gráficas de caja.
figIy = px.box(data_df, y='Iy')
figIy.show()
figPF = px.box(data_df, y='PF')
figPF.show()
fige = px.box(data_df, y='e')
fige.show()
figdIf = px.box(data_df, y='dIf')
figdIf.show()
figIf = px.box(data_df, y='If')
figIf.show()

'''/* 
Function: IQR_outlier

Confirma de manera numérica la existencia de outliers.

Parameters:

    data_df - Datos recopilados del CSV.
    
Returns:

    Un DataFrame que contiene si un dato se sale de los límites calculados.
    Si un valor es igual a NaN significa que no es un outliers.
*/'''

def IQR_outlier(data_df):
    
    # Guarda el resultado del cuartil del 25%
    q1 = data_df.quantile(0.25)
    # Guarda el resultado del cuartil del 75%
    q3 = data_df.quantile(0.75)
    
    # Calcula el rango entre cuantiles
    IQR = q3-q1 
    
    ''' 
    Revisa si hay datos afuera de cada límite.
    Los limites se calculan de la siguiente manera: 
        Límite superior: q3 + (1.5*IQR)
        Límite inferior: q1 - (1.5*IQR)
    '''
    
    outliers = data_df[((data_df<(q1-1.5*IQR)) | (data_df>(q3+1.5*IQR)))]

    return outliers

# Iprime el resultado de la función anterior.
print(IQR_outlier(data_df))

,Iy,PF,e,dIf,If
count,557.000000,557.000000,557.000000,557.000000,557.000000
mean,4.499820,0.825296,0.174704,0.350659,1.530659
std,0.896024,0.103925,0.103925,0.180566,0.180566
min,3.000000,0.650000,0.000000,0.037000,1.217000
25%,3.700000,0.740000,0.080000,0.189000,1.369000
50%,4.500000,0.820000,0.180000,0.345000,1.525000
75%,5.300000,0.920000,0.260000,0.486000,1.666000
max,6.000000,1.000000,0.350000,0.769000,1.949000


Iy     0
PF     0
e      0
dIf    0
If     0
dtype: int64

     Iy  PF   e  dIf  If
0   NaN NaN NaN  NaN NaN
1   NaN NaN NaN  NaN NaN
2   NaN NaN NaN  NaN NaN
3   NaN NaN NaN  NaN NaN
4   NaN NaN NaN  NaN NaN
..   ..  ..  ..  ...  ..
552 NaN NaN NaN  NaN NaN
553 NaN NaN NaN  NaN NaN
554 NaN NaN NaN  NaN NaN
555 NaN NaN NaN  NaN NaN
556 NaN NaN NaN  NaN NaN

[557 rows x 5 columns]


# Separación de los set de datos y normalizacón
### Se va a usar una función de activación sigmoide

In [78]:
'''/* 
Esta sección prepara los datos recopilados anteriormente en subsecciones para entrenamieto y prueba.
Además de esto, normaliza los datos para poder ser ingresados a una función sigmoide.
*/'''

# Arreglo que contiene los datos de prueba dados por el profresor.
tarea_data = [[3, 0.64, 0.01, 0.031, 0], [4.38, 0.78, 0.14, 0.402, 0], [6, 1.01, 0.35,0.799, 0]]
# Convierte un arreglo en DataFrame con las columnas llamadas: 'Iy', 'PF', 'e', 'dIf'.
tarea_data = pd.DataFrame(tarea_data, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
cp = tarea_data
display(cp)

# Guarda las características del conjunto de datos antes de normalizar.
train_stats = data_df.describe()
tarea_stats = tarea_data.describe()

# Se transpone el DataFrame para que las columnas y filas se inviertan entre ellas.
train_stats = train_stats.transpose()
tarea_stats = tarea_stats.transpose()

display(tarea_stats)

#display(train_stats)
#display(data_df)

'''/* 
Function: norm

Normaliza los datos a través de una función min-max entre 0 y 1 para ser alimentados a una función sigmoide.

Parameters:

    x - Datos a normalizar.
    
Returns:

    Los datos alimentados ya normalizados.
*/'''

def norm(x, stats):
    return((x-stats['min'])/(stats['max']-stats['min'])) #min-max norm

'''/* 
Function: reverse_norm

Desnormaliza los datos a través de la funcion inversa de la min-max.

Parameters:

    x - Datos normalizados a revertir.
    
Returns:

    Los datos alimentados ya desnormalizados.
*/'''

def reverse_norm(x):
    return(x*(train_stats['max']-train_stats['min'])+train_stats['min'])

# Normaliza todo el conjunto de datos
data_df = norm(data_df, train_stats)
# Mueve 80% de los datos en un subconjuto de entrenamiento y los aleatoriza.
train_df = data_df.sample(frac=0.8, random_state=0)
# Mueve el restante de los datos en un subconjuto de prueba y los aleatoriza.
test_df = data_df.drop(train_df.index)
# Guarda los datos de la comlumna If en una variable para las etiquetas de entrenamiento.
train_labels = train_df.pop('If')
# Guarda los datos de la comlumna If en una variable para las etiquetas de prueba.
test_labels = test_df.pop('If')

tarea_data = norm(tarea_data, tarea_stats)
tarea_df = tarea_data.drop(['If'], axis=1)
display(tarea_df)

,Iy,PF,e,dIf,If
0,3.00,0.64,0.01,0.031,0
1,4.38,0.78,0.14,0.402,0
2,6.00,1.01,0.35,0.799,0


,count,mean,std,min,25%,50%,75%,max
Iy,3.0,4.460000,1.501599,3.000,3.6900,4.380,5.1900,6.000
PF,3.0,0.810000,0.186815,0.640,0.7100,0.780,0.8950,1.010
e,3.0,0.166667,0.171561,0.010,0.0750,0.140,0.2450,0.350
dIf,3.0,0.410667,0.384073,0.031,0.2165,0.402,0.6005,0.799
If,3.0,0.000000,0.000000,0.000,0.0000,0.000,0.0000,0.000


,Iy,PF,e,dIf
0,0.00,0.000000,0.000000,0.000000
1,0.46,0.378378,0.382353,0.483073
2,1.00,1.000000,1.000000,1.000000


# Creación del modelo

In [79]:
'''/* 
Function: my_model

Genera el modelo de regresión MLP y lo compila. El modelo consiste de lo siguiente:
    - Una capa de entradas de forma train_df.keys() que es igual a la cantidad de características.
    - Una capa oculta con dos neuronas y como función de activación una función sigmoide.
    - Una capa oculta con dos neuronas y como función de activación una función sigmoide.
    - Una capa de salida con una neurona y como función de activación una función sigmoide.
    - Utiliza ADAM como optimizador.
    - La función de pérdida es la Suma Cuadrática del Error.
    
Parameters:

    my_learning_rate - Razón de aprendizaje.
    
Returns:

    El modelo neuronal ya creado.
*/'''

def my_model(my_learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2, input_shape=[len(train_df.keys())], activation='sigmoid'),
        tf.keras.layers.Dense(2, activation='sigmoid'),#se hace una hidden layer de 3 neuronas con activacion sigmoid
        tf.keras.layers.Dense(1, activation='sigmoid') #capa de output
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss="mean_squared_error", #funcion de perdida
                metrics=[tf.keras.metrics.MeanSquaredError()] #funcion de metricas para evaluar
                )
    
    return model

model = my_model(0.01)
model.summary()

example_batch = train_df[:10]
example_result = model.predict(example_batch)
example_result

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 2)                 10        
                                                                 
 dense_13 (Dense)            (None, 2)                 6         
                                                                 
 dense_14 (Dense)            (None, 1)                 3         
                                                                 
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 40ms/step


array([[0.4530725 ],
       [0.4528775 ],
       [0.45411107],
       [0.45188513],
       [0.45192796],
       [0.4514849 ],
       [0.45367435],
       [0.45293507],
       [0.45298547],
       [0.45424733]], dtype=float32)

# Entrenamiento del modelo

In [80]:
'''/* 
Function: train_model

Alimenta los datos correspondientes a las características y etiquetas al modelo y conduce el proceso de entrenamiento y validación.
Además de esto, también le establece al modelo la cantidad de ciclos de entrenamiento y el tamaño del batch.
Por último crea un subconjunto de validación con un 25% de los datos de entrenamiento.
    
Parameters:

    model - El modelo creado anteriormente.
    features - Conjunto que contiene las características.
    labels - Conjunto que contiene las etiquetas.
    epochs - Cantidad de ciclos de entrenamiento
    batch_size - Tamaño del batch.
    
Returns:

    Los resultados del entrenamiento.
*/'''

def train_model(model, features, labels, epochs, batch_size):
    early_stop= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) #patience es la cant de epochs antes de chequear el improvement
    history = model.fit(
        x=features,
        y=labels,
        epochs=epochs,
        batch_size = batch_size,
        validation_split=0.25,
        callbacks = early_stop
    )
    
    # Guarda los resultados obtenidos del proceso de entrenamiento y validación en un DataFrame.
    # Estos resultados son las pérdidas y el número del ciclo correspondiente.
    hist= pd.DataFrame(history.history)
    # Añade al DataFrame la información sobre los ciclos.
    hist['epoch'] = history.epoch
    # Guarda los datos de error en una variable para uso futuro.
    mse = hist['mean_squared_error']
    
    return hist, mse

# Ploteo de gráficas

In [81]:
'''/* 
Function: plot_loss_curve

Grafica las curvas de pérdida correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''

def plot_loss_curve(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de pérdida por una versión más legible.  
    labels = {"mean_squared_error":"Training Loss", "val_mean_squared_error":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Mean Square Error", "variable":"Curvas de Pérdida"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()

'''/* 
Function: plot_predictions

Grafica las etiquetas de prueba contra las predicciones hechas por la red neuronal.
    
Parameters:

    predictions - Datos por graficar.
    
Returns:

    La gráfica ya creada.
*/'''

def plot_predictions(predictions):
    
    # Se crea una traza de puntos.
    trace1 = go.Scatter(
        x = predictions['If'],
        y = predictions['If Predictions'],
        name = 'Predicciones',
        mode='markers',
    )
    
    # Se crea una traza de línea.
    trace2 = go.Line(
        x= predictions['If'],
        y = predictions['If'],
        name = 'Datos Reales',
        yaxis='y2'
    )
    
    # Se crea la figura 
    fig = make_subplots(specs=[[{"secondary_y": True}]], x_title='If', y_title='Predicción de If')
    # Se le agrega un título.
    fig.update_layout(title_text="Gráfica De Predicciones Contra Datos Reales")
    # Se le agrega la primera traza.
    fig.add_trace(trace1)
    # Se le agrega la segunda traza.
    fig.add_trace(trace2,secondary_y=False)
    fig.update_layout(template='plotly_white')
    
    fig.show()
    

# Hyperparámetros

In [82]:
learning_rate = 0.01
epochs = 100
batch_size = 7

# Llamado de funciones

In [83]:
# Llama a la función para crear el modelo y lo guarda.
model = my_model(learning_rate)
# Invoca a la función de entrenamiento y guarda los resultados.
history, mse = train_model(model, train_df, train_labels, epochs, batch_size)

Epoch 1/100
48/48 [==============================] - 1s 5ms/step - loss: 0.0831 - mean_squared_error: 0.0831 - val_loss: 0.0644 - val_mean_squared_error: 0.0644
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 0.0582 - mean_squared_error: 0.0582 - val_loss: 0.0624 - val_mean_squared_error: 0.0624
Epoch 3/100
48/48 [==============================] - 0s 1ms/step - loss: 0.0549 - mean_squared_error: 0.0549 - val_loss: 0.0587 - val_mean_squared_error: 0.0587
Epoch 4/100
48/48 [==============================] - 0s 1ms/step - loss: 0.0502 - mean_squared_error: 0.0502 - val_loss: 0.0515 - val_mean_squared_error: 0.0515
Epoch 5/100
48/48 [==============================] - 0s 1ms/step - loss: 0.0424 - mean_squared_error: 0.0424 - val_loss: 0.0409 - val_mean_squared_error: 0.0409
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 0.0316 - mean_squared_error: 0.0316 - val_loss: 0.0282 - val_mean_squared_error: 0.0282
Epoch 7/100
48/48 [===============

In [84]:
# Llama a la función de las gráficas.
plot_loss_curve(history)
display(history)

,loss,Training Loss,val_loss,Validation Loss,epoch
0,0.083130,0.083130,0.064447,0.064447,0
1,0.058188,0.058188,0.062430,0.062430,1
2,0.054942,0.054942,0.058747,0.058747,2
3,0.050152,0.050152,0.051525,0.051525,3
4,0.042362,0.042362,0.040913,0.040913,4
...,...,...,...,...,...
95,0.000150,0.000150,0.000129,0.000129,95
96,0.000143,0.000143,0.000133,0.000133,96
97,0.000152,0.000152,0.000151,0.000151,97
98,0.000149,0.000149,0.000178,0.000178,98


# Predicciones

In [85]:
# Hace predicciones usando el conjunto de datos de prueba.
test_predictions = model.predict(test_df).flatten()
tarea_predictions = model.predict(tarea_df).flatten()

# Crea una arreglo de ceros con las dimensiones forma (111, 4) lo cuál es la cant de filas y de columnas -1 de los datos del CSV.
pivot1 = np.zeros((111,4))
# En la columna que falta se le agregan los labels de test normalizados.
pivot1 = np.insert(pivot1, 4, test_labels, axis =1)

# Crea un arreglo de ceros con las dimensiones de los datos del CSV.
pivot2 = np.zeros((111,5))

#El ciclo cambia los ceros en la última columna por los datos de la predicción.
for i in range(111):
    for j in range(5):
        pivot2[i,4] = test_predictions[i]

# Guarda las predicciones en un DataFrame.
test_predictions_df1 = pd.DataFrame(pivot1, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
test_predictions_df2 = pd.DataFrame(pivot2, columns=['Iy', 'PF', 'e', 'dIf', 'If'])

# Desnormaliza las predicciones y los labels.
test_predictions_df1 = reverse_norm(test_predictions_df1)
test_predictions_df2 = reverse_norm(test_predictions_df2)

# Renombra la columna "If" por "If Predictions" en el dataframe de las predicciones
test_predictions_df2.rename(columns = {'If':'If Predictions'}, inplace = True)
test_predictions_df = pd.concat([test_predictions_df1['If'], test_predictions_df2['If Predictions']], axis= 1)

# Llama a la función que grafica las predicciones sobre los datos.
plot_predictions(test_predictions_df)
    
rms= mean_squared_error(test_predictions_df1['If'],test_predictions_df2['If Predictions'])
print(rms)
    
display(test_predictions_df)


pivot3 = np.zeros((len(tarea_predictions),5))
for i in range(len(tarea_predictions)):
        for j in range(5):
            pivot3[i,4] = tarea_predictions[i]
            
pivot_df = pd.DataFrame(pivot3, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
pivot_df = reverse_norm(pivot_df)

display(pivot_df['If'])

1/1 [==============================] - 0s 16ms/step


C:\Users\dave4\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




5.7422864721409854e-05


,If,If Predictions
0,1.563,1.562166
1,1.401,1.399544
2,1.490,1.492553
3,1.373,1.368375
4,1.498,1.499735
...,...,...
106,1.647,1.644493
107,1.369,1.366476
108,1.862,1.855055
109,1.893,1.872243


0    1.281318
1    1.649764
2    1.581832
Name: If, dtype: float64

# Ceteris Paribus

In [86]:
# Se crean arreglos con una columna variable y con números del 2 al 4 para cambiar esos espacios por constantes.
# Los 5 son para cambiarlos por valores con ruido aleatorio de 5%, 15% o 25%

# Arreglo dónde la columna 0 es variable, corresponde a la variable Iy
arr1 = np.array([[0, 2, 3, 4], [0.46, 2, 3, 4], [1, 2, 3, 4], 
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4],
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4],
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4]])

# Arreglo dónde la columna 1 es variable, corresponde a la variable PF
arr2 = np.array([[2, 0, 3, 4], [2, 0.378378, 3, 4], [2, 1, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4]])

# Arreglo dónde la columna 2 es variable, corresponde a la variable e
arr3 = np.array([[2, 3, 0, 4], [2, 3, 0.382353, 4], [2, 3, 1, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4]])

# Arreglo dónde la columna 3 es variable, corresponde a la variable dIf
arr4 = np.array([[2, 3, 4, 0], [2, 3, 4, 0.483073], [2, 3, 4, 1],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5]])

# Reemplazan los 2 con un float aleatorio entre 0 y 1 
arr1[arr1 == 2] = np.random.random()
arr2[arr2 == 2] = np.random.random()
arr3[arr3 == 2] = np.random.random()
arr4[arr4 == 2] = np.random.random()

# Reemplazan los 3 con un float aleatorio entre 0 y 1
arr1[arr1 == 3] = np.random.random()
arr2[arr2 == 3] = np.random.random()
arr3[arr3 == 3] = np.random.random()
arr4[arr4 == 3] = np.random.random()

# Reemplazan los 4 con un float aleatorio entre 0 y 1
arr1[arr1 == 4] = np.random.random()
arr2[arr2 == 4] = np.random.random()
arr3[arr3 == 4] = np.random.random()
arr4[arr4 == 4] = np.random.random()


'''/* 
Function: complete

Cambia los 5 en los arreglos con cualquiera de los 3 primeros datos con un ruido aleatorio de 5%, 15% o 25%.
Manteniendo el rango entre 0 y 1.
    
Parameters:

    arr - Arreglo de datos.
    
Returns:

    El arreglo con 12 valores diferentes para la variable en evaluación.
*/'''

def complete(arr):
    porcentajes = [0.05, 0.15, 0.25]
    valores = [0,1,2]
    for i in range(len(arr)):
        for j in range(4):
            if ((arr[i][j] == 5)):
                rand1 = random.randint(0, 2)
                valor = valores[rand1]
                rand2 = random.randint(0, 2)
                porcentaje = porcentajes[rand2]
                if(valor != 2):
                    arr[i][j] = arr[valor][j] + arr[valor][j]*porcentaje
                else:
                    arr[i][j] = arr[valor][j] - arr[valor][j]*porcentaje
    return arr

# Guarda en arreglo creado por la función complete en otro con el nombre de la variable con la que se trabaja.
iy = complete(arr1)
pf = complete(arr2)
e = complete(arr3)
dif = complete(arr4)

# Predice los valores para cada uno de los arreglos
iy_pred = model.predict(iy)
pf_pred = model.predict(pf)
e_pred = model.predict(e)
dif_pred = model.predict(dif)

'''/* 
Function: cp_plot

Grafica las predicciones hechas contra a la variable en evaluación.
    
Parameters:

    data - Datos por graficar.
    var - Nombre de la variable en evaluación.
    
Returns:

    La gráfica ya creada.
*/'''

def cp_plot(data, var):
    # Crea la figura, establece los títulos de eje y los rangos de los ejes
    fig = px.line(data, x='x', y='y',
                labels={"x": "{}".format(var), "y":"Predicción"}, range_y=[0,1], range_x=[0,1])
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()
    
# Genera el dataframe con los datos necesarios para hacer las gráficas.
graph_iy = pd.DataFrame({'x':iy[0:12,0].reshape((12)),'y':iy_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_pf = pd.DataFrame({'x':pf[0:12,1].reshape((12)),'y':pf_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_e = pd.DataFrame({'x':e[0:12,2].reshape((12)),'y':e_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_dif = pd.DataFrame({'x':dif[0:12,3].reshape((12)),'y':dif_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)

# Se crean las gráficas.
cp_plot(graph_iy, 'Iy')
cp_plot(graph_pf, 'PF')
cp_plot(graph_e, 'e')
cp_plot(graph_dif, 'dIf')

1/1 [==============================] - 0s 21ms/step


In [87]:
best_model = tf.keras.models.load_model('Modelos_regresion/m7')
best_model.summary()

#loss, acc = best_model.evaluate(, data_labels, verbose=2)
#print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

#model.save('Modelos_regresion/modelo1')

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 4)                 20        
                                                                 
 dense_43 (Dense)            (None, 4)                 20        
                                                                 
 dense_44 (Dense)            (None, 1)                 5         
                                                                 
Total params: 45
Trainable params: 45
Non-trainable params: 0
_________________________________________________________________
